In [1]:
import os
import time
import torch
import itertools
import networkx as nx
from collections import defaultdict
from torch_geometric.datasets import TUDataset
from torch_geometric.utils import from_networkx, subgraph

# Graphlet Kernel Implementation

In [2]:
def graphlet_features(data, k=3):
    # NOTE - might need to add sampling instead of enumeration, to deal with snap data
    features = {}
    nodes = list(range(data.num_nodes))
    for subset in itertools.combinations(nodes, k):
        subset = list(subset)
        sub_edge_index, _ = subgraph(subset, data.edge_index, relabel_nodes=True, num_nodes=data.num_nodes)
        if sub_edge_index.size(1) == 0:
            continue
        
        sub_edge_index_np = sub_edge_index.cpu().numpy()
        unique_edges = set()
        for i in range(sub_edge_index_np.shape[1]):
            u, v = sub_edge_index_np[0, i], sub_edge_index_np[1, i]
            if u > v:
                u, v = v, u
            unique_edges.add((u, v))
        
        if len(unique_edges) < k - 1:
            continue
        
        degrees = [0] * k
        for u, v in unique_edges:
            degrees[u] += 1
            degrees[v] += 1
        label = tuple(sorted(degrees))
        features[label] = features.get(label, 0) + 1
    return features

def graphlet_kernel(data1, data2, k=3):

    feat1 = graphlet_features(data1, k)
    feat2 = graphlet_features(data2, k)
    
    keys = set(feat1.keys()) | set(feat2.keys())
    
    vec1 = torch.tensor([feat1.get(key, 0) for key in keys], dtype=torch.float32)
    vec2 = torch.tensor([feat2.get(key, 0) for key in keys], dtype=torch.float32)
    
    kernel_value = torch.dot(vec1, vec2)
    return kernel_value

In [3]:
G_ba = nx.barabasi_albert_graph(n=50, m=4)
G_ws = nx.watts_strogatz_graph(n=50, k=4, p=0.1)

data_ba = from_networkx(G_ba)
data_ws = from_networkx(G_ws)

kernel_synthetic = graphlet_kernel(data_ba, data_ws, k=3)
print("Graphlet Kernel between BA and WS synthetic graphs:", kernel_synthetic.item())


Graphlet Kernel between BA and WS synthetic graphs: 289001.0


In [4]:
def wl_features(data, h=2):
    if hasattr(data, 'x') and data.x is not None:
        labels = [str(tuple(x.tolist())) for x in data.x]
    else:
        labels = ['1'] * data.num_nodes

    feature_dict = defaultdict(int)
    for lab in labels:
        feature_dict[lab] += 1

    label_lookup = {}
    next_label_id = 0

    def compress(label_str):
        nonlocal next_label_id
        if label_str not in label_lookup:
            label_lookup[label_str] = str(next_label_id)
            next_label_id += 1
        return label_lookup[label_str]

    for it in range(h):
        new_labels = [None] * data.num_nodes
        for v in range(data.num_nodes):
            neighbors = data.edge_index[1, data.edge_index[0] == v].tolist() if data.edge_index.size(0) > 0 else []
            multiset = sorted([labels[u] for u in neighbors])
            new_label_str = labels[v] + "_" + "_".join(multiset)
            new_labels[v] = compress(new_label_str)
            feature_dict[new_labels[v]] += 1
        labels = new_labels  

    return feature_dict

def wl_kernel(data1, data2, h=2):
    feat1 = wl_features(data1, h)
    feat2 = wl_features(data2, h)
    keys = set(feat1.keys()) | set(feat2.keys())
    vec1 = torch.tensor([feat1.get(key, 0) for key in keys], dtype=torch.float32)
    vec2 = torch.tensor([feat2.get(key, 0) for key in keys], dtype=torch.float32)
    return torch.dot(vec1, vec2)

In [7]:
file_path = 'snap_dataset/facebook_combined.txt'

G_public = nx.read_edgelist(file_path, nodetype=int)
print(f"Loaded public graph with {G_public.number_of_nodes()} nodes and {G_public.number_of_edges()} edges.")
data_public = from_networkx(G_public)
kernel_value = graphlet_kernel(data_public, G_ba, k=3)
print("Graphlet Kernel (public graph vs. itself):", kernel_value.item())

Loaded public graph with 4039 nodes and 88234 edges.


KeyboardInterrupt: 

In [5]:
dataset = TUDataset(root='/tmp/TUDataset', name='MUTAG')

data1 = dataset[0]
data2 = dataset[1]

kernel_val_wl = wl_kernel(data1, data2, h=2)
print("WL Kernel (Graph 0 vs. Graph 1):", kernel_val_wl.item())

num_graphs = len(dataset)
wl_kernel_matrix = torch.zeros((num_graphs, num_graphs))
wl_features_list = [wl_features(data, h=2) for data in dataset]

all_labels = set()
for feat in wl_features_list:
    all_labels.update(feat.keys())

def features_to_vector(feat, keys):
    return torch.tensor([feat.get(key, 0) for key in keys], dtype=torch.float32)

all_labels = list(all_labels) 
feature_vectors = [features_to_vector(feat, all_labels) for feat in wl_features_list]

for i in range(num_graphs):
    for j in range(i, num_graphs):
        k_val = torch.dot(feature_vectors[i], feature_vectors[j])
        wl_kernel_matrix[i, j] = k_val
        wl_kernel_matrix[j, i] = k_val

print("WL Kernel Matrix shape:", wl_kernel_matrix.shape)


WL Kernel (Graph 0 vs. Graph 1): 211.0
WL Kernel Matrix shape: torch.Size([188, 188])


In [6]:
kernel_val_graphlet = graphlet_kernel(data1, data2, k=3)
kernel_val_wl = wl_kernel(data1, data2, h=2)

print("Graphlet Kernel (Graph 0 vs. Graph 1):", kernel_val_graphlet.item())
print("WL Kernel (Graph 0 vs. Graph 1):", kernel_val_wl.item())

start_time = time.time()
_ = wl_kernel(data1, data2, h=2)
wl_time = time.time() - start_time

start_time = time.time()
_ = graphlet_kernel(data1, data2, k=3)
graphlet_time = time.time() - start_time

print(f"WL Kernel time: {wl_time:.6f} seconds")
print(f"Graphlet Kernel time: {graphlet_time:.6f} seconds")

Graphlet Kernel (Graph 0 vs. Graph 1): 513.0
WL Kernel (Graph 0 vs. Graph 1): 211.0
WL Kernel time: 0.000824 seconds
Graphlet Kernel time: 0.031891 seconds
